In [3]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

from mle_logging import MLELogger

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Storing Logging Results

If you provide a .json file path and a seed_id, the log will be created in a sub-directory.

Furthermore, the .json file will be copied for reproducibility.

Multiple simultanous runs (different seeds) can now log to the same directory. Everything else remains the same.

In [4]:
# Instantiate logging to experiment_dir
log_seed_0 = MLELogger(time_to_track = ['num_updates', 'num_epochs'],
                       what_to_track = ['train_loss', 'test_loss'],
                       experiment_dir = "multi_seed_dir/",
                       config_fname = "base_config.json",  # Provide path to .json config
                       seed_id = "seed_0",                 # Provide some seed identifier (str)   
                       use_tboard = True,
                       model_type = 'torch')

log_seed_1 = MLELogger(time_to_track = ['num_updates', 'num_epochs'],
                       what_to_track = ['train_loss', 'test_loss'],
                       experiment_dir = "multi_seed_dir/",
                       config_fname = "base_config.json",  # Provide path to .json config
                       seed_id = "seed_1",                 # Provide some seed identifier (str)   
                       use_tboard = True,
                       model_type = 'torch')

In [5]:
# Save some time series statistics
time_tic = {'num_updates': 10,
            'num_epochs': 1}
stats_tic = {'train_loss': 0.1234,
             'test_loss': 0.1235}

# Update the log with collected data & save it to .hdf5
log_seed_0.update(time_tic, stats_tic, save=True)
log_seed_1.update(time_tic, stats_tic, save=True)

# Aggregating the Two Seed Logs

In [9]:
import os
from mle_logging.merge import merge_hdf5_files

log_dir = "multi_seed_dir/21-08-04_base_config/logs/"
merge_hdf5_files(new_filename = os.path.join(log_dir, "seed_aggregated.hdf5"),
                 log_paths = [os.path.join(log_dir, "21-08-04_base_config_seed_0.hdf5"),
                              os.path.join(log_dir, "21-08-04_base_config_seed_1.hdf5")])

In [ ]:
!pwd